# Probar la búsqueda en Azure AI Search

### Generar los embedings para una consulta

In [ ]:
#%pip install azure-search-documents
#%pip install azure-identity

import os
import dotenv
import openai
import json

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)


In [ ]:
openai_client = openai.OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"]
)
MODEL_NAME = "text-embedding-3-small"

In [ ]:
query_input = "Resume la hoja de vida de Diego Zumárraga Mera"
embeddings_response = openai_client.embeddings.create(
    model=MODEL_NAME,
    input=query_input,
)
query_embedding = embeddings_response.data[0].embedding
print(query_embedding)

### Buscar la consulta en Azure AI Search

In [ ]:
item = {"query": query_input,
        "embedding": query_embedding
}

In [ ]:
search_client = SearchClient(
        endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
        index_name="my-index",
        credential=DefaultAzureCredential(),
    )

In [ ]:
results = []
vector_query = VectorizedQuery(
            vector=item["embedding"], k_nearest_neighbors=3, fields="embedding"
        )
result = search_client.search(
    search_text=item["query"],
    vector_queries=[vector_query],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=2,
)

### Obtener la mejor respuesta

In [ ]:
answers = result.get_answers()
if answers:
    print("Answers:")
    for answer in answers:
        print(f"Answer: {answer.text}")
        print(f"Confidence: {answer.score}")

### Obtener todos los resultados

In [ ]:
docs = [
        {
            "id": doc["id"],
            "title": doc["title"],
            "content": doc["content"],
            "rerankerScore": doc["@search.reranker_score"],
        }
        for doc in result
    ]

# Remove duplicates
results.extend([i for i in docs if i["id"] not in [x["id"] for x in results]])

print(f"Found {len(results)} results")
for i, doc in enumerate(results):
    print(f"Result {i + 1}:")
    print(f"Title: {doc['title']}")
    print(f"Content: {doc['content']}")
    print(f"Reranker Score: {doc['rerankerScore']}")
    print()